In [1]:
# !pip install groq qdrant-client
# !pip install langchain-groq
# !pip install langchainhub

# Imports

In [1]:
import os
from PyPDF2 import PdfReader
import numpy as np
from langchain_community.vectorstores import Qdrant
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
import numpy as np
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import Tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.llms import OpenAI
from langchain_groq import ChatGroq
import gradio as gr


# Extracting Text from PDFs

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    Extract text content from a PDF file.

    Args:
    - pdf_path (str): The path to the PDF file.

    Returns:
    - str: The extracted text content.
    """
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for page in reader.pages:
        extracted_text += page.extract_text()
    return extracted_text

def extract_text_from_pdfs_in_directory(directory):
    """
    Extract text content from all PDF files in a directory and save as text files.

    Args:
    - directory (str): The path to the directory containing PDF files.
    """
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory, filename)
            extracted_text = extract_text_from_pdf(pdf_path)
            txt_filename = os.path.splitext(filename)[0] + ".txt"
            txt_filepath = os.path.join(directory, txt_filename)
            with open(txt_filepath, "w") as txt_file:
                txt_file.write(extracted_text)

# Specify the directory containing PDF files
directory_path = "Docs/"

# Extract text from PDFs in the directory and save as text files
extract_text_from_pdfs_in_directory(directory_path)


# Generating Embedding 

In [3]:
GROQ_API_KEY = "<GROQ_API_KEY>"

collection_name = "legal_contract"

In [4]:
directory_path = "Docs"

# List all .txt files in the directory
txt_files = [file for file in os.listdir(directory_path) if file.endswith('.txt')]

loader = TextLoader(directory_path + "/" + txt_files[0])
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [5]:
# Initialize the TextEmbedding model
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

In [6]:
retriever = qdrant.as_retriever()

# Setting ReAct Agents

In [7]:
# convert PDR retriever into a tool
tool_search = create_retriever_tool(
    retriever,
    "eligibility",
    "Checks for eligibility of the contract",
)

In [8]:
tool_search.func

functools.partial(<function _get_relevant_documents at 0x702e960f2cb0>, retriever=VectorStoreRetriever(tags=['Qdrant', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x702e93fd1030>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')

In [9]:
tool_search.args_schema.schema()

{'title': 'RetrieverInput',
 'description': 'Input to the retriever.',
 'type': 'object',
 'properties': {'query': {'title': 'Query',
   'description': 'query to look up in retriever',
   'type': 'string'}},
 'required': ['query']}

In [10]:
def get_age(name: str, person_database: dict) -> int:
    """
    Get the age of a person from the database.

    Args:
    - name (str): The name of the person.
    - person_database (dict): A dictionary containing person information.

    Returns:
    - int: The age of the person if found, otherwise None.
    """
    if name in person_database:
        return person_database[name]["Age"]
    else:
        return None

def get_health(name: str, person_database: dict) -> str:
    """
    Get the health status of a person from the database.

    Args:
    - name (str): The name of the person.
    - person_database (dict): A dictionary containing person information.

    Returns:
    - str: The health status of the person if found, otherwise None.
    """
    if name in person_database:
        return person_database[name]["Health"]
    else:
        return None

def get_age_info(name: str) -> str:
    """
    Get age and health information for a person.

    Args:
    - name (str): The name of the person.

    Returns:
    - str: A string containing age and health information for the person.
    """
    person_database = {
        "Sam": {"Age": 21, "Nationality": "US", "Health": "Good"},
        "Alice": {"Age": 25, "Nationality": "UK", "Health": "Good"},
        "Bob": {"Age": 11, "Nationality": "US", "Health": "Mentally Not Sound"}
    }
    age = get_age(name, person_database)
    if age is not None:
        return f"\nAge: {age}\n"
    else:
        return f"\nAge Information for {name} not found.\n"
    
def get_health_info(name: str) -> str:
    """
    Get age and health information for a person.

    Args:
    - name (str): The name of the person.

    Returns:
    - str: A string containing age and health information for the person.
    """
    person_database = {
        "Sam": {"Age": 21, "Nationality": "US", "Health": "Good"},
        "Alice": {"Age": 25, "Nationality": "UK", "Health": "Mentally Not Sound"},
        "Bob": {"Age": 11, "Nationality": "US", "Health": "Good"}
    }
    health = get_health(name, person_database)
    if health is not None:
        return f"\nHealth: {health}\n"
    else:
        return f"\nHealth Information for {name} not found.\n"

# Define the Tool
get_age_info_tool = Tool(
    name="Get Age",
    func=get_age_info,
    description="Useful for getting age information for any person. Input should be the name of the person."
)
get_health_info_tool = Tool(
    name="Get Health",
    func=get_health_info,
    description="Useful for getting health information for any person. Input should be the name of the person."
)


# Common Agent prompts

In [11]:
prompt_react = hub.pull("hwchase17/react")
print(prompt_react.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


# Running the ReAct agent

In [12]:
# tools = [tool_search, get_info_tool]

tools = [get_age_info_tool, get_health_info_tool]

model = ChatGroq(model_name="llama3-70b-8192", groq_api_key=GROQ_API_KEY, temperature=0)
# model = OpenAI(openai_api_key="<YOUR_OPENAI>")

react_agent = create_react_agent(model, tools=tools, prompt=prompt_react)
react_agent_executor = AgentExecutor(
    agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

/home/yash/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [14]:
# Eligibility Check Example:
query = "I am Bob. Can i make a legal contract?"
react_agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: To determine if Bob can make a legal contract, I need to know if Bob is of legal age. In most places, the legal age for entering into contracts is 18 years old.

Action: Get Age
Action Input: Bob
Age: 11
Thought: Since Bob is only 11 years old, he is not of legal age to enter into a contract. In most places, the legal age for entering into contracts is 18 years old.

Final Answer: No, Bob cannot make a legal contract.

> Finished chain.


{'input': 'I am Bob. Can i make a legal contract?',
 'output': 'No, Bob cannot make a legal contract.'}

In [15]:
# Query for retrieving personal information
query = "What is Alice's age?"
react_agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
I need to find out Alice's age.

Action: Get Age
Action Input: Alice
Age: 25
I now know the final answer
Final Answer: Alice is 25 years old.

> Finished chain.


{'input': "What is Alice's age?", 'output': 'Alice is 25 years old.'}

In [13]:
# Query for eligibility check
query = "I am Alice. Can I make a legal contract?"
react_agent_executor.invoke({"input": query})




> Entering new AgentExecutor chain...
 Since the question involves a person's name, the best tools to use would be Get Age and Get Health.
Action: Get Age
Action Input: Alice
Age: 25
25 years old is generally considered to be old enough to make a legal contract, but it's always best to check the person's health status as well.
Action: Get Health
Action Input: Alice
Health: Mentally Not Sound
 Based on Alice's health status, it may not be advisable for her to make a legal contract. It would be best to consult a legal professional for further guidance.
Final Answer: It is uncertain whether Alice can make a legal contract due to her mental health status. It would be best to consult a legal professional for further guidance.

> Finished chain.


{'input': 'I am Alice. Can I make a legal contract?',
 'output': 'It is uncertain whether Alice can make a legal contract due to her mental health status. It would be best to consult a legal professional for further guidance.'}

In [18]:
import gradio as gr
from io import StringIO
import sys
import re

def generate_response(question):
    """
    Generate a response based on the provided question using ChatGroq.

    Args:
    - question (str): The question input by the user.

    Returns:
    - str: The generated response based on the question.
    """
    tools = [get_age_info_tool, get_health_info_tool]

    model = ChatGroq(model_name="llama3-70b-8192", groq_api_key=GROQ_API_KEY, temperature=0)
    # model = OpenAI(openai_api_key="<YOUR_OPENAI>")

    react_agent = create_react_agent(model, tools=tools, prompt=prompt_react)
    react_agent_executor = AgentExecutor(
        agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True
    )
    
    # Redirect stdout to capture text
    with StringIO() as text_output:
        sys.stdout = text_output
        completion = react_agent_executor.invoke({"input": question})
        sys.stdout = sys.__stdout__  # Reset stdout
        
        # Get the captured text
        text_output_str = text_output.getvalue()
    
    # Remove ANSI escape codes
    text_output_str = re.sub(r'\x1b\[[0-9;]*m', '', text_output_str)

    return text_output_str

# Set up the Gradio interface
iface = gr.Interface(
    fn=generate_response,
    inputs=[gr.Textbox(label="Question")],  # Pass input as a list
    outputs=[gr.Textbox(label="Generated Response")],  # Pass output as a list
    title="Intellegent RAG with Qdrant, LangChain ReAct and Llama3 from Groq Endpoint",
    description="Enter a question and get a generated response based on the retrieved text.",
)

iface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.24.0, however version 4.29.0 is available, please upgrade.
--------


/home/yash/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
